In [ ]:
# default_exp modelos

# modelos

> Herramientas de ajuste, validación y evaluación de modelos

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import numpy as np
import pandas as pd
import geopandas as gpd
import statsmodels.formula.api as smf 
import statsmodels.api as sm
from criminologia_cdmx.etl import *
from criminologia_cdmx.covariables import *

## variable_independiente

In [ ]:
# export
def variable_independiente(datos, columna_y, valor_y, fecha_inicio, fecha_fin, agregacion='colonias'):
    """ Regresa un DataFrame con la variable independicente agregada entre fecha_inicio y fecha_fin
        en las unidades requeridas.
        
        Args:
            datos (DataFrame): carpetas/victimas con ids espaciales y categorías de usuario
            columna_y (str): Nombre de la columna en donde vienen los incidentes de valor_y
            valor_y (str): delito o categoría a utilizar como Y
            fecha_inicio (str): fecha inicial para agregar delitos "d-m-Y"
            fecha_fin (str): fecha final para agregar delitos "d-m-Y"
            agregacion (str): colonias/cuadrantes. Eventualmente debe recibir 
                              agregaciones arbitrarias
    """
    fecha_inicio = pd.to_datetime(fecha_inicio, dayfirst=True)
    fecha_fin = pd.to_datetime(fecha_fin, dayfirst=True)
    datos = datos.loc[datos['fecha_hechos'].between(fecha_inicio, fecha_fin)]
    datos = datos.loc[datos[columna_y] == valor_y]
    if agregacion == 'colonias':
        columna_agrega = 'colonia_cve'
        layer = 'colonias'
    elif agregacion == 'cuadrantes':
        columna_agrega = 'cuadrante_id'
        layer = 'cuadrantes'
    else:
        raise ValueError("unidades debe ser 'colonias' o 'cuadrantes'")
    datos = datos.groupby(columna_agrega).size()
    datos.name = valor_y
    unidades = gpd.read_file("datos/criminologia_capas.gpkg", layer=layer)
    datos = unidades[[columna_agrega]].merge(datos, on=columna_agrega, how='left').fillna(0)
    return datos   

In [ ]:
carpetas = get_carpetas_from_api(100000)
carpetas = agrega_ids_espaciales(carpetas)
fecha_inicio = carpetas.fecha_hechos.min().strftime("%d-%m.%Y")
fecha_fin = carpetas.fecha_hechos.max().strftime("%d-%m.%Y")
delito = 'ROBO A CASA HABITACION SIN VIOLENCIA'
Y = variable_independiente(carpetas, 'delito', delito, fecha_inicio, fecha_fin, 
                           agregacion='cuadrantes')
assert delito in Y.columns
Y = variable_independiente(carpetas, 'delito', delito, fecha_inicio, fecha_fin)
assert delito in Y.columns
Y

,colonia_cve,ROBO A CASA HABITACION SIN VIOLENCIA
0,32,0.0
1,619,2.0
2,1792,0.0
3,1774,0.0
4,583,1.0
...,...,...
1819,1731,2.0
1820,1801,0.0
1821,1799,0.0
1822,1800,1.0


## CapaDeAnalisis

In [ ]:
# export
class CapaDeAnalisis(object):
    """ Clase para contener variable objetivo y covariables.
    
        Args:
            Y (DataFrame): debe tener dos columnas: el identificador de la unidad de análisis y 
                           el valor de la variable dependiente 
                           (las columnas deben venir en ese orden).
            covariables (DataFrame): debe contener una columna con el identificador de la unidad 
                                     de análisis (común a Y) y tantas como covariables.
            campo_id (str): el nombre del campo común en X y Y para unirlos.
        Atributos:
            Y (DataFrame): la variable dependiente.
            Y_nombre (str): Nombre de la columna con el delito a modelar.
            X (DataFrame): las variables independientes.
            X_nombres (list): Lista de los nombres de columnas de las covariables.
            campo_id (str): el nombre del campo común en X y Y para unirlos.
            df (DataFrame): la unión de los dos anteriores.
    """
    
    def __init__(self, Y, covariables, campo_id):
        self.Y = Y
        self.Y_nombre = Y.columns[-1]
        self.X = covariables
        self.X_nombres = [x for x in covariables.columns if x != campo_id]
        self.campo_id = campo_id
        self.df = self.__merge_covars()
        
    def __merge_covars(self):
        df = (self.Y.merge(self.X, on=self.campo_id)
                    .replace([np.inf, -np.inf], np.nan)
                    .dropna())
        return df

    # TODO: 
    # agregar/quitar variables
    # checar que exista el campo_id en las dos bases
    # quitamos unas filas, hay que llevar registro de eso
    # Calcular variables con retraso espacial
    

In [ ]:
usos = get_uso_de_suelo()
usos = agrega_uso_suelo(usos, unidades='colonias')
ca = CapaDeAnalisis(Y, usos, 'colonia_cve')
assert ca.X.equals(usos)
assert ca.Y.equals(Y)

## ModeloGLM

In [ ]:
# export
class ModeloGLM(object):
    """ Wrapper para modelos de Regresión GLM de statsmodels.
        
        La clase prepara y ajusta un modelo GLM usando la variable objetivo
        definida en CapaDeAnalisis y **todas** las covariables.
        
        Args:
            capa (CapaDeAnalisis): objeto con las variables del modelo.
            familia (statsmodels.api.families.Family()) la distribución a usar en el modelo GLM.
            
        Atributos:
            capa (CapaDeAnalisis): objeto con las variables del modelo.
            familia (statsmodels.api.families.Family()) la distribución a usar en el modelo GLM.
            formula (str): la fórmula usada para ajustar el modelo.
            df_resultado (DataFrame): los resultados como DataFrame.
            df_diagnostico (DataFrame): los diagnósticos como DataFrame.
        Métodos:
            fit(): Ajusta el modelo.
    """
    def __init__(self, capa, familia):
        self.capa = capa
        self.familia = familia
        self.formula = self.__get_formula()
        self.__modelo = self.__get_modelo()
        self.df_resultado = None
        self.df_diagnostico = None
        
    def __get_formula(self):
        ls = f"Q('{self.capa.Y_nombre}') ~ "
        rs = ""
        for nombre in self.capa.X_nombres:
            rs += f"Q('{nombre}') + "        
        formula = ls + rs
        formula = formula[:-3]
        return formula
    
    def __get_modelo(self):
        modelo = smf.glm(formula = self.formula,
                         data    = self.capa.df, 
                         family  = self.familia)
        return modelo
    
    def __resultados_a_df(self, resultados):
        """LLena self.df_resultado con los resultados del ajuste."""
        results_df = pd.DataFrame({"coef":resultados.params,
                                   "std err": resultados.bse,
                                   "z": resultados.tvalues,
                                   "P>|z|":resultados.pvalues,
                                   "conf_lower":resultados.conf_int()[0],
                                   "conf_higher":resultados.conf_int()[1]
                                    })
        self.df_resultado = results_df

    def __diagnostico_a_df(self, resultados):
        """LLena self.df_diagnostico con los resultados del ajuste."""
        indice = ["Log-Likelihood", "Deviance", "Pearson chi2"]
        valores = [resultados.llf, resultados.deviance, resultados.pearson_chi2]
        results_df = pd.DataFrame({"Diagnóstico": indice, "Valor": valores})
        self.df_diagnostico = results_df
        
    def fit(self):
        fm = self.__modelo.fit()
        self.__resultados_a_df(fm)
        self.__diagnostico_a_df(fm)
        return fm

In [ ]:
m = ModeloGLM(ca, sm.families.NegativeBinomial())
fm = m.fit()
assert type(m.df_resultado) == pd.DataFrame
assert type(m.df_diagnostico) == pd.DataFrame